Importing packages


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import numpy as np
import pandas as pd
import seaborn as sns
# Using plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode()

# **UNITED KINGDOM SEARCH TRENDS**

Let's Look At The Dataset first

In [ ]:
data=pd.read_csv('../input/coronavirus-top-cities-and-trending-searches/United Kingdom.csv')
data.fillna('',inplace=True)
data.head()

**Looks like a lot of usable information but not in a much usable format.Let's do some transformations to make it usable for further analysis!**

  Extract the towns and country names

In [ ]:
columns=list(data.columns)

def get_info(text):
    a=text.split(',')
    return {'town': a[0],'country':a[1]}
def get_date(text):
    b=text.split(' ')
    return b[0]
for i in range(0,len(columns),2):
    data[columns[i]][1:]=data[columns[i]][1:].apply(lambda x:get_info(x))
for i in range(0,len(columns)):
    data[columns[i]][0]=get_date(data[columns[i]][0])
data.head()    

> Getting the list of the **unique** towns,country and dates from our dataset for further transformation

In [ ]:
country=[]
for i in range(0,len(columns),2):
    for j in range(1,data.shape[0]):
        country.append(data[columns[i]][j]['country'])

country=list(set(country))

town=[]
for i in range(0,len(columns),2):
    for j in range(1,data.shape[0]):
        town.append(data[columns[i]][j]['town'])

l=list(set(town))

dates=[]
for i in range(len(columns)):
    dates.append(data[columns[i]][0])

dates=list(set(dates))

# **TOWN-BASED Transformation**

**Transforming the dataset into a more usable format in which the Trending queries corresponding to a particular town and date are clearly identified.**

In [ ]:
import itertools
data_=pd.DataFrame(columns=dates,index=l)
c = list(itertools.product(dates, l))
for i in c:
    for j in range(0,len(columns),2):
        if data[columns[j]][0]==i[0]:
            trend=[]
            for k in range(1,data.shape[0]):
                if data[columns[j]][k]['town']==i[1]:
                    trend.append(data[columns[j+1]][k])
                    
            data_[i[0]][i[1]]=trend
                    
            
for j in data_.columns:
    
    data_[j] = data_[j].apply(lambda y: np.nan if y==[] or y==[''] else y)

In [ ]:
data_

In [ ]:
df=data_.dropna(how='all',axis=0)
df.isnull().sum()
df=df[df.columns[df.isnull().sum()<470]]
df.fillna('',inplace=True)

In [ ]:
df=df.transpose()
df

# **TOWN BASED MONTHLY SEARCH TRENDS ANALYSIS**

The resulting dataset **df_monthly_** gives us the list of all the trending queries corresponding to the respective towns in July,August and September.


In [ ]:
df['Date']=df.index
df['Month_Name']=pd.DatetimeIndex((df['Date'])).month_name()
df.fillna('',inplace=True)
list_df=[]
for i in list(df.columns[:-2]):
    df_monthly=df.groupby('Month_Name').agg({i: lambda x: list(x)}).reset_index()
    list_df.append(df_monthly) 
df_monthly_=pd.concat(list_df,axis=1,join='inner').drop('Month_Name',axis=1)
df_monthly_['Month']=df_monthly['Month_Name']
for i in list(df.columns[:-2]):
    df_monthly_[i]=df_monthly_[i].apply(lambda t:[item for sublist in t for item in sublist])
category=list(df.columns[:-2])

df_monthly_

# **Plotly Visualization of the monthly Town wise trends.**

* Here you can select the town from the dropdown to get the corresponding top searches 
* The slider can be used to toggle between the months and see the trends

Note:For some instances the graph shows no searches ranked as they were the null values in the dataset

In [ ]:
fig = go.Figure()

for i in range(len(df_monthly_['Month'])):
    fig.add_trace(
        go.Bar(
            visible=False,
            x=['1','2','3','4','5'], y=[2.5,2.5,2.5,2.5,2.5],
            text=list(pd.Series(np.array(df_monthly_['Oadby'][i])).value_counts()[:5].index)
            ))
fig.data[0].visible = True

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Townwise Search Trends: " + str(df_monthly_['Month'][i])},
              ],  # layout attribute     
        label = df_monthly_['Month'][i]     
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Month: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_traces(textposition='outside',hoverinfo='text')
fig.update_traces(marker_color='white')
fig.update_layout(plot_bgcolor='yellow')
fig.update_traces(textfont_size=40)

fig.update_yaxes(showticklabels=False, range=[0,8])
fig.update_xaxes(showticklabels=False)

fig.update_layout(
    sliders=sliders,
)

button_dict =[]
for i in range(len(category)):
    button = dict(
        args=["text",[
            list(pd.Series(np.array(df_monthly_[category[i]][0])).value_counts()[:5].index),
            list(pd.Series(np.array(df_monthly_[category[i]][1])).value_counts()[:5].index),
            list(pd.Series(np.array(df_monthly_[category[i]][2])).value_counts()[:5].index),
            
                     ]],
        label=category[i],
        method="restyle"
    )
    button_dict.append(button)

fig.update_layout(
    updatemenus=[
        dict(
            buttons= button_dict,
            direction="down",
            pad={"l": 10, "t": 10},
            showactive=True,
            x=0.25,
            xanchor="left",
            y=1.15,
            yanchor="top"
            ),
    ],
)
fig.update_layout(annotations=[
                    dict(text='1', x=0, y=1.5, font_size=15, showarrow=False,bgcolor="#ff7f0e"),
                    dict(text='2', x=1, y=1.5, font_size=15, showarrow=False,bgcolor="#ff7f0e"),
                    dict(text='3', x=2, y=1.5, font_size=15, showarrow=False,bgcolor="#ff7f0e"),
                    dict(text='4', x=3, y=1.5, font_size=15, showarrow=False,bgcolor="#ff7f0e"),
                    dict(text='5', x=4, y=1.5, font_size=15, showarrow=False,bgcolor="#ff7f0e"),

])

fig.update_traces(opacity = .9)
fig.update_layout(title="Google Search Trends: August",height=500)


fig.show()

# **A Similar Trend-Analysis Based on the Countries this time**


Country Based Transformation Of Dataset

In [ ]:
import itertools
data_2=pd.DataFrame(columns=dates,index=country)
c = list(itertools.product(dates, country))
for i in c:
    for j in range(0,len(columns),2):
        if data[columns[j]][0]==i[0]:
            trend=[]
            for k in range(1,data.shape[0]):
                if data[columns[j]][k]['country']==i[1]:
                    if data[columns[j+1]][k]!='':
                        trend.append(data[columns[j+1]][k]  )
                    
            data_2[i[0]][i[1]]=trend
                    
for j in data_2.columns:
    data_2[j] = data_2[j].apply(lambda y: np.nan if y==[] or y==[''] else y)
df_=data_2.transpose()
df_

# **COUNTRY BASED MONTHLY SEARCH TRENDS ANALYSIS**

In [ ]:
df_['Date']=df_.index
df_['Month_Name']=pd.DatetimeIndex((df_['Date'])).month_name()
df_.fillna('',inplace=True)
df_monthly_1=df_.groupby('Month_Name').agg({' Northern Ireland': lambda x: list(x)}).reset_index()
df_monthly_2=df_.groupby('Month_Name').agg({' Scotland': lambda x: list(x)}).reset_index()
df_monthly_3=df_.groupby('Month_Name').agg({' Wales': lambda x: list(x)}).reset_index()
df_monthly_4= df_.groupby('Month_Name').agg({' England': lambda x: list(x)}).reset_index()
df_monthly=pd.concat([df_monthly_3,df_monthly_2,df_monthly_1,df_monthly_4],axis=1,join='inner').drop('Month_Name',axis=1)
df_monthly['Month']=df_monthly_1['Month_Name']
df_monthly[' England']=df_monthly[' England'].apply(lambda t:[item for sublist in t for item in sublist])
df_monthly[' Wales']=df_monthly[' Wales'].apply(lambda t:[item for sublist in t for item in sublist])
df_monthly[' Scotland']=df_monthly[' Scotland'].apply(lambda t:[item for sublist in t for item in sublist])
df_monthly[' Northern Ireland']=df_monthly[' Northern Ireland'].apply(lambda t:[item for sublist in t for item in sublist])
category=[' England', ' Wales', ' Scotland', ' Northern Ireland']

The resulting dataset df_monthly_ gives us the list of all the trending queries corresponding to the respective COUNTRIES in July,August and September.

In [ ]:
df_monthly

# **Plotly Visualization of the monthly Country wise trends.**

In [ ]:
fig = go.Figure()

for i in range(len(df_monthly['Month'])):
    fig.add_trace(
        go.Bar(
            visible=False,
            x=['1','2','3','4','5'], y=[2.5,2.5,2.5,2.5,2.5],
            text=list(pd.Series(np.array(df_monthly[' England'][i])).value_counts()[:5].index)
            ))
fig.data[0].visible = True

steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": "Country Wise Search Trends: " + str(df_monthly['Month'][i])},
              ],  # layout attribute     
        label = df_monthly['Month'][i]     
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Month: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_traces(textposition='outside',hoverinfo='text')
fig.update_traces(marker_color='white')
fig.update_layout(plot_bgcolor='yellow')
fig.update_traces(textfont_size=40)

fig.update_yaxes(showticklabels=False, range=[0,8])
fig.update_xaxes(showticklabels=False)

fig.update_layout(
    sliders=sliders,
)

button_dict =[]
for i in range(len(category)):
    button = dict(
        args=["text",[
            list(pd.Series(np.array(df_monthly[category[i]][0])).value_counts()[:5].index),
            list(pd.Series(np.array(df_monthly[category[i]][1])).value_counts()[:5].index),
            list(pd.Series(np.array(df_monthly[category[i]][2])).value_counts()[:5].index),
            
                     ]],
        label=category[i],
        method="restyle"
    )
    button_dict.append(button)

fig.update_layout(
    updatemenus=[
        dict(
            buttons= button_dict,
            direction="down",
            pad={"l": 10, "t": 10},
            showactive=True,
            x=0.25,
            xanchor="left",
            y=1.15,
            yanchor="top"
            ),
    ],
)
fig.update_layout(annotations=[
                    dict(text='1', x=0, y=1.5, font_size=15, showarrow=False,bgcolor="#ff7f0e"),
                    dict(text='2', x=1, y=1.5, font_size=15, showarrow=False,bgcolor="#ff7f0e"),
                    dict(text='3', x=2, y=1.5, font_size=15, showarrow=False,bgcolor="#ff7f0e"),
                    dict(text='4', x=3, y=1.5, font_size=15, showarrow=False,bgcolor="#ff7f0e"),
                    dict(text='5', x=4, y=1.5, font_size=15, showarrow=False,bgcolor="#ff7f0e"),

])


fig.update_traces(opacity = .9)
fig.update_layout(title="Google Search Trends: August",height=500)


fig.show()

# **BITERM TOPIC MODELLING FOR ENGLAND SEARCH QUERIES**


This model is accurate in short text classification. It explicitly models the word co-occurrence patterns in the whole corpus to solve the problem of sparse word co-occurrence at document-level.

**We will classify the queries in 10 topics using a biterm topic model and specify the topics of the queries in the dataframe.**

In [ ]:
!pip install biterm
import nltk

In [ ]:
a=[]
for i in df_[' England']:
    for j in i:
        a.append(j)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from biterm.utility import vec_to_biterms, topic_summuary
stpwrd = nltk.corpus.stopwords.words('english')
new_stopwords=['coronavirus','uk','cases','deaths','news','today']
stpwrd.extend(new_stopwords)
vec = CountVectorizer(stop_words=stpwrd)
X = vec.fit_transform(a).toarray()
from biterm.utility import vec_to_biterms
vocab = np.array(vec.get_feature_names())
biterms = vec_to_biterms(X)
from biterm.cbtm import oBTM

btm = oBTM(num_topics=10, V=vocab)
topics = btm.fit_transform(biterms, iterations=90)


In [ ]:
topic_summuary(btm.phi_wz.T, X, vocab, 20)

In [ ]:
for i in range(len(a)):
    print("{} (topic: {})".format(a[i], topics[i].argmax()))

In [ ]:
topic=[]
for i in df_[' England']:
    x=[]
    for j in i:
        for k in range(len(a)):
            if j==a[k]:
                x.append(topics[k].argmax())
    topic.append(list(set(x)))

In [ ]:
final_df=pd.DataFrame(index=df_.index)

final_df['England']=df_[' England']
final_df['Topic']=pd.Series(np.array(topic,dtype='object')).values
final_df